# CNN Facial Recognition

# Introduction
Hello! My selfgiven name is the Stats Whisper and I'm back with another Data Science topic that is one of the hottest things in research now: Deep Learning. Even if you lend a very small ear to what is going on in the scientific community, you have probably heard of Deep Learning. It is the backbone of some of the coolest things today like: self-driving cars, facial-recongition, virtual digital assistants and [accurately estimating how a human on the other side of a wall is standing/sitting/walking just from perturbations in Wifi signals caused by that human](http://rfpose.csail.mit.edu/) (don't know what good that is for but hey).


Believe or not, neural networks have been around for a while but the slashed costs of computing power paired with the increased computing capacity are the impetus behind the eponential growth of the field. 

With the expantion of the field grew different kinds of neural networks for different purposes. 
Today, we will be focusing on a particular type of neural network called a convolution neural network (CNN for short). Research has found these models to be excellent when it comes to image recognition. 

Okay, so before we dive into what exactly a convulution neural network is let us strip it down to its essence, just a plain and simple vanilla neural network. Like the depths of the universe, you can go as far, wide and deep into this topic and you'll probably never get to the bottom of it because of the presence of a very active area of research that is pushing the field even further. 

So what exactly is a neural network? Well, it's actually pretty simple: a function. A function that can be very complex but a function nevertheless. 

At its most basic core, a neural network has 3 parts:
 - input layer
 - hidden layer
 - output layer

As the name suggests, the input layer is where the data is feed into the network. The mysterious hidden layer is where all the action is happening and output layer is the result you wish to acquire. 

In relation to a function, take the ubiquitous function of simple line found in every Algebra class: y = mx+b. The x in this case is the input layer. The m and b are hidden layer and the y is the output layer. 

Easy right?

A convolution neural network does the exact same thing by using an image as input, crunches some numbers then does a prediction as an output.

There is a great amount of content out there that goes into good detail and further explains how the mechanics of a convolution neural network work. It can get very mathematical and technical really quick so you can easily lose an audience. But if you are really interested to go further, I found [this video](https://www.youtube.com/watch?v=aircAruvnKk) to be a lifesaver for my Big Data class while I was learning neural networks in grad school.



## Application

If you ever wondered how in the world your new Iphone has the capacity to determine whether or not it's you or someone else's face with amazing accuracy to unlock your phone using Face ID, well you have just witnessed the power of neural networks first hand. Apple trained the neural networks using billions of images then took the result and installed it into your iphone. Pretty cool, huh?

Since neural networks are great at determining who's face that image belongs to, what if we could weild this technology find me my celebrity doublegangger. 


### Loading the Data

Okay so first let's load the data. We'll be using Google's Colab product to leverage the powerfull GPU available, slashing runtimes instead waiting forever for it to run locally on my MacBook.



In [ ]:
#loading google drive where data is stored to run it on Colab
from google.colab import drive
drive.mount('/content/gdrive')

#importing the necesary libraries and modules to compile data for ingest by CNN.
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import TensorBoard
K.set_image_dim_ordering('th')
from time import time
batch_size = 64

train_datagen = ImageDataGenerator(data_format="channels_last") 

test_datagen = ImageDataGenerator(data_format="channels_last")

train_generator = train_datagen.flow_from_directory(
        '/content/gdrive/My Drive/data/train',  # this is the target directory on google drive
        target_size=(197, 197),  # all images will be resized to 197x197
        batch_size=batch_size,
        color_mode='rgb',
        class_mode='categorical') 

# this is a similar generator, for validation data
validation_generator = test_datagen.flow_from_directory(
        '/content/gdrive/My Drive/data/validation',
        target_size=(197, 197),
        batch_size=batch_size,
        color_mode='rgb',
        class_mode='categorical')

tensorboard = TensorBoard(log_dir="tensorBoard/".format(time()), write_images=True, write_graph=True)


### Transfer Learning for CNN

Instead of creating a CNN from scratch and having to use valuable time and resources training/testing different models, what we can do instead is use a model that has already been trained then slightly customize it to fit our objectives. The formal term for this technique is called "transfer learning". 

Transfer learning is great because you can take neural networks and their respective weights painstakingly developed by the pros then reuse those models for other purposes. 

While TensorFlow is the gold standard for neural networks I perfer to use Keras because of the way it simplyfies long cumbersome TensorFlow code into a few simple lines of code and, more importantly, Keras comes pre-trained models available for execute transfer learning. 

Today, we'll be using a CNN developed by researchers at Cornell University called [MobileNetV2](https://arxiv.org/abs/1801.04381). While most of the available models have compariable accuracy rates, this model is "lightweight" with it's fewer total parameters to train and thus faster to train. 

In [ ]:
#importing the relevant libraries and modules for CNN
from keras.applications.mobilenet_v2 import MobileNetV2
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D, Input, Dropout
from keras import backend as K
import keras

input_tensor = Input(shape=(197, 197, 3))

# create the base pre-trained model
base_model = MobileNetV2(input_tensor=input_tensor, weights='imagenet', include_top=False)


# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(512, activation='relu')(x)
# since we have 3 classes, we need a final layer that predicts 3 classes. 
predictions = Dense(3, activation='softmax')(x)

custom_model = Model(inputs=base_model.input, outputs=predictions)

# compile the model
custom_model.compile(optimizer=keras.optimizers.Adam(lr=0.0001), loss='categorical_crossentropy',metrics=['accuracy'])


custom_model.fit_generator(
        train_generator,
        steps_per_epoch=10,
        epochs=10,
        validation_data=validation_generator,
        validation_steps=10)

So now with a fully trained model